In [51]:
import random

# Ejercicio N°1sdsdsds
Se tiene un RDD con el registro de notas de los alumnos de la forma (padrón, materia, nota,
fecha). Se pide resolver utilizando PySpark:

- A. Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años.
- B. Un RDD conteniendo el promedio de notas de cada alumno de la forma (padrón,
promedio).
- C. El nombre y apellido del alumno con mejor promedio. Para esto puede utilizarse un
segundo RDD alumnos con registros (padron, nombre y apellido).


In [2]:
notas = []
padrones = []
cant_alumnos = 50
for i in range(cant_alumnos):
    padron = random.randint(99900,100000)
    while(padron in padrones): padron = random.randint(99900,100000)
    padrones.append(padron)
    materia = random.choice(["AM3","AM2","TALLER","LABORATORIO","DATOS"])
    nota = random.randint(1,10)
    fecha = "{}-{}-{}".format(random.choice(['2017','2018','2019']),
                                        random.randint(1,12),
                                        random.randint(1,28))
    tupla = (padron,materia,nota,fecha)
    notas.append(tupla)


In [3]:
notasRDD = sc.parallelize(notas)

In [4]:
#A
notasRDD.filter(lambda x: x[3] >= '2018-01-01' and x[3]<= '2019-12-31' and x[2]>=4)\
        .count()  

14

In [5]:
#B
promediosRDD = notasRDD.map(lambda x: (x[0], (x[2],1)))\
                       .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
                       .map(lambda x: (x[0], x[1][0]/x[1][1]))

In [6]:
alumnos = []
for i in range(cant_alumnos):
    padron = padrones[i]
    nombre = random.choice(["Gustavo","Nicolas","Mariana","Florencia","Ayelen","Pedro","Gonzalo"])
    apellido = random.choice(["Alvarez","Fernandez","Gonzalez","Ramirez","Rodriguez"])
    alumno = (padron, nombre, apellido)
    alumnos.append(alumno)

In [7]:
alumnosRDD = sc.parallelize(alumnos)

In [8]:
#C
alumnosRDD.map(lambda x: (x[0], (x[1] + ' ' + x[2])))\
          .join(promediosRDD)\
          .map(lambda x: (x[1][0], x[1][1]))\
          .reduce(lambda x,y: x if x[1]>=y[1] else y)

('Mariana Alvarez', 10.0)

# Ejercicio N°2
Se tiene un RDD registros de ventas de producto con la forma (fecha de venta, código de
producto, precio de venta) y en otro RDD detalle de los productos con (código de producto,
descripción del producto, categoría). Se pide resolver utilizando PySpark:
- A. Cuál es el producto más vendido.
- B. Cuál es la categoría de productos más vendida.
- C. Cuál es el top5 de productos más vendidos generando un RDD con (código de
producto, descripción, cantidad de ventas)
- D. Cuál es el producto que registró mayor aumento de precio en el último año, tomando
para este análisis solo los productos que cuenten con al menos 50 ventas en el último
año.
- E. Idem anterior, pero calculando la categoría de productos que registró mayor variación
de precios en el último año.

In [9]:
ventas = []
productos = []

for i in range(10000):
    fecha_de_venta = "{}-{}-{}".format(random.choice(["2018","2019"])
                                                     ,random.choice(["01","02","03","04","05","06","07","08","09","10","11","12"])
                                                     ,random.randint(1,28))
    codigo_de_producto = random.randint(0,10)
    precio_de_venta = round(random.uniform(100,5000),2)
    venta = (fecha_de_venta, codigo_de_producto, precio_de_venta)
    ventas.append(venta)
for i in range(0,10):
    codigo_de_producto = i
    descripcion = "descripcion" + ' ' + str(i)
    categoria = random.choice(["Limpieza","Producto de cocina", "Vestimenta", "Juguete"])
    producto = (codigo_de_producto, descripcion, categoria)
    productos.append(producto)

In [10]:
ventasRDD = sc.parallelize(ventas)
productosRDD = sc.parallelize(productos)

In [11]:
#A - (Codigo_producto, cantidad_vendida)
ventasRDD.map(lambda x: (x[1],1))\
         .reduceByKey(lambda x,y: x+y)\
         .reduce(lambda x,y: x if x[1]>y[1] else y)

(5, 944)

In [12]:
#B
cant = ventasRDD.map(lambda x: (x[1],1))\
                .reduceByKey(lambda x,y: x+y)
#cant es un RDD con las cantidades de venta de cada prod [(id_producto, cant)]
productosRDD.map(lambda x: (x[0], x[2]))\
            .join(cant)\
            .map(lambda x: (x[1][0], x[1][1]))\
            .reduce(lambda x,y: x if x[1] >= y[1] else y ) 
#Me devuelve la tupla (categoria,cant), puedo hacer [0] y obtener la categoria

('Limpieza', 944)

In [13]:
#C
productosRDD_map = productosRDD.map(lambda x: (x[0], x[1]))
top5RDD = sc.parallelize(cant.top(5,key = lambda x: x[1]))\
            .join(productosRDD_map)\
            .map(lambda x: (x[0],x[1][1],x[1][0]))
top5RDD.count()

4

In [14]:
top5RDD.takeOrdered(5,key = lambda x: -x[2])

[(5, 'descripcion 5', 944),
 (2, 'descripcion 2', 936),
 (9, 'descripcion 9', 922),
 (3, 'descripcion 3', 914)]

In [15]:
#D
#Ult año me va a servir para poder filtrar los que tengan mas de 50 ventas en el ultimo año. 
ult_año = ventasRDD.filter(lambda x: x[0] >= "2018-01" and x[0] <= "2019-01")\
                   .map(lambda x: (x[1],1))\
                   .reduceByKey(lambda x,y: x+y)\
                   .filter(lambda x: x[1] >= 50)

ventas_sep = ventasRDD.map(lambda x: (x[1], (x[0],x[2])))\
                      .join(ult_año)\
                      .map(lambda x: (x[1][0][0], x[0], x[1][0][1]))\
                      .map(lambda x: (0,(x[1],x[2])) if (x[0] >= "2018-01" and x[0] <= "2018-02")\
                        else ((1,(x[1],x[2])) if (x[0]>="2018-12" and x[0]<= "2019-01") else None))\
                      .filter(lambda x: x!= None)\
                      .map(lambda x: ((x[0],x[1][0]), x[1][1]))\
                      .reduceByKey(lambda x,y: x if x>y else y)


In [16]:
def porcent(x):
    porc = (x[1][1]*100/x[1][0]) - 100
    return round(porc,2)

In [17]:
ventas_primer_mes = ventas_sep.filter(lambda x: x[0][0] == 0).map(lambda x: (x[0][1], x[1]))
ventas_segund_mes = ventas_sep.filter(lambda x: x[0][0] == 1).map(lambda x: (x[0][1], x[1]))
ventas_primer_mes.join(ventas_segund_mes)\
                               .map(lambda x: (x[0], porcent(x)))\
                               .top(1,key = lambda x: x[1])

[(2, 5.17)]

In [18]:
#E
productosRDD_map = productosRDD.map(lambda x: (x[0], x[2]))

ult_año = ventasRDD.filter(lambda x: x[0] >= "2018-01" and x[0] <= "2019-01")\
                   .map(lambda x: (0,(x[1],x[2])) if (x[0] >= "2018-01" and x[0] <= "2018-02")\
                        else ((1,(x[1],x[2])) if (x[0]>="2018-12" and x[0]<= "2019-01") else None))\
                   .filter(lambda x: x!= None)\
                   .map(lambda x: (x[1][0], (x[1][1], x[0] )))\
                   .join(productosRDD_map)\
                   .map(lambda x: ( (x[1][1],x[1][0][1]),x[1][0][0]))\
                   .reduceByKey(lambda x,y: x+y)

ventas_primer_mes = ult_año.filter(lambda x: x[0][1] == 0).map(lambda x: (x[0][0], x[1]))
ventas_segund_mes = ult_año.filter(lambda x: x[0][1] == 1).map(lambda x: (x[0][0], x[1]))
ventas_primer_mes.join(ventas_segund_mes)\
                 .map(lambda x: (x[0], porcent(x)))\
                 .top(1,key = lambda x: x[1])


[('Producto de cocina', 27.22)]

# Ejercicio N°3

Se tiene un RDD con información de vuelos programados con la forma (número de vuelo, código de aerolínea, código de aeropuerto de salida, código de aeropuerto de llegada, fecha de salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM). 

A su vez, se cuenta con el registro actualizado del estado de los vuelos que fueron ocurriendo, con la forma (número de vuelo, aerolínea, fecha de salida AAAAMMDD, hora de salida HH:MM, fecha de llegada AAAAMMDD, hora de llegada HH:MM, estado).

En base al estado, podría contar con algún dato en blanco, por ejemplo si el vuelo fue cancelado no tendrá información de fechas y horas, si el vuelo se encuentra aún en curso, no contendrá información de la llegada. Se pide resolver utilizando PySpark:

- A. Cuál es el aeropuerto con mayor tránsito.
- B. Cuál es la aerolínea con mayor cantidad de vuelos.
- C. Cuál es la aerolínea con mayor cantidad de cancelaciones.
- D. Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de salida.
- E. Cuál es el vuelo (numero de vuelo + fecha) con mayor retraso en el horario de llegada.
- F. Cuál es la aerolínea más puntual.
- G. Cuál es el aeropuerto que registra mayor desviación con - respecto a los horarios coordinados.

In [1]:
from datetime import datetime

In [11]:
vuelos = [(1,1,'MDQ','EZ', '20191214', '15:30', '20191214', '17:30')]
estados = [(1, 'Aerolineas', '20191214', '15:45', '20191214', '18:00', 3)]
vuelosRDD = sc.parallelize(vuelos)
estadosRDD = sc.parallelize(estados)

In [12]:
#A
vuelosRDD.map(lambda x: ((x[0],x[3]), 1 ))\
         .reduceByKey(lambda x,y: x+y)\
         .map(lambda x: (x[0][1],x[1]))\
         .top(1, key= lambda x: x[1])

[('EZ', 1)]

In [13]:
#B
vuelosRDD.map(lambda x: ((x[0],x[1]),1))\
         .reduceByKey(lambda x,y: x+y)\
         .map(lambda x: (x[0][1],x[1]))\
         .top(1, key=lambda x: x[1])

[(1, 1)]

#C
estadosRDD.filter(lambda x: x[6] == "Cancelado")\
          .map(lambda x: (x[1],1))\
          .reduceByKey(lambda x,y: x+y)\
          .top(1,key=lambda x: x[1])

In [47]:
#D
horario_ideal = vuelosRDD.map(lambda x: ((x[0],x[4]), datetime.strptime(x[5], "%H:%M").time()))
horario_real = estadosRDD.map(lambda x: ((x[0],x[2]), datetime.strptime(x[3], "%H:%M").time()))
horario_ideal.join(horario_real)\
             .map(lambda x: ([0][0], x[1][0] - x[1][1])).collect()
             .top(1, key=lambda x: x[1])

[(1, datetime.time(15, 30))]

In [26]:
#E
horario_ideal = vuelosRDD.map(lambda x: ((x[0],x[6]), datetime.strptime(x[7], "%H:%M").time()))
horario_real = estadosRDD.map(lambda x: ((x[0],x[4]), datetime.strptime(x[5], "%H:%M").time()))
horario_ideal.join(horario_real)\
             .map(lambda x: ([0][0], x[1][0] - x[1][1])).collect()
             .top(1, key=lambda x: x[1])

[((1, '20191214'), datetime.time(15, 30))]

In [48]:
#F


# Ejercicio N°4
Se tiene un RDD con las coordenadas de rectángulos de la forma (x1,x2,y1,y2). Se pide
programar en PySpark un programa que encuentre el rectángulo de superficie mínima que
contiene al punto (w,z)

In [56]:
rectangulos = []

for i in range(100):
    x1 = random.randint(1,50)
    x2 = random.randint(2,51)
    if x2 <= x1: x2 = random.randint(2,51)
    y1 = random.randint(1,50)
    y2 = random.randint(2,51)
    if y2 <= y1: y2 = random.randint(2,51)
    rectangulos.append( (x1,x2,y1,y2) )
    
rectangulosRDD = sc.parallelize(rectangulos)

In [71]:
w = 10
z = 10

In [76]:

rectangulosRDD.filter(lambda x: w >= x[0] and w <= x[1] and z >= x[2] and z <= x[3])\
              .map(lambda x: ((x[0],x[1],x[2],x[3]),abs(x[1] - x[0] * (x[3] - x[2]))))\
              .top(1, key = lambda x: -x[1])

[((3, 41, 5, 16), 8)]

# Ejercicio N°5
Se tiene un RDD con libros en donde cada registro es un texto. Se pide obtener todos los
anagramas de mas de 7 letras que puedan encontrarse. El formato de salida debe ser una lista
de listas en donde cada lista tiene un conjunto de palabras que son anagramas. Ejemplo:
[[discounter,introduces,reductions],[percussion,supersonic]...]

In [218]:
def generate_key(word):
    rep = []
    word = ','.join(word).split(',')
    word = sorted(word)
    for i,letter in enumerate (word):
        if(letter == word[i-1]): continue
        rep.append(word.count(letter))
    return (word, rep)

In [219]:
def separador(text):
    reg = []
    words = text.split(' ')
    for word in words: 
        if(len(word) < 7): continue
        key = generate_key(word.lower())
        reg.append((key,word))
    return reg    

In [220]:
import re

In [236]:
cuentos = ["Aaadhlmo Aaadhlom Aaadhmlo Aaadhmol Aaadholm Aaadhoml Aaadlhmo Aaadlhom Aaadlmho Aaadlmoh Aaadlohm Aaadlomh Aaadmhlo Aaadmhol almohada","Estaban introduces acaso a la puerta discounter dos mujeres mozas,percussion destas que supersonic llaman del partido, las cuales iban a Sevilla con unos arrieros que en la venta aquella noche acertaron a hacer jornada; y, como a nuestro aventurero todo cuanto pensaba, veía o imaginaba le parecía ser hecho y pasar al modo de lo que había leído, luego que vio la venta, se le representó que era un castillo con sus cuatro torres y chapiteles de luciente plata, sin faltarle su puente levadiza y honda cava, con", "lalalala"]
librosRDD = sc.parallelize(cuentos)

In [237]:
llaves = librosRDD.map(lambda x: " ".join(re.findall("[a-zA-Z]+", x)))\
         .map(lambda x: separador(x)).collect()
palabras = []
for i in llaves:
    for llave in i:
        palabras.append(llave)

In [239]:

palabrasRDD = sc.parallelize(palabras)
palabrasRDD.map(lambda x: ((tuple(x[0][0]), tuple(x[0][1])), x[1]))\
           .reduceByKey(lambda x,y: x+ ',' +y)\
           .map(lambda x: x[1]).collect()

['mujeres',
 'partido',
 'aquella',
 'jornada',
 'nuestro',
 'aventurero',
 'pensaba',
 'castillo',
 'luciente',
 'levadiza',
 'lalalala',
 'Aaadhlmo,Aaadhlom,Aaadhmlo,Aaadhmol,Aaadholm,Aaadhoml,Aaadlhmo,Aaadlhom,Aaadlmho,Aaadlmoh,Aaadlohm,Aaadlomh,Aaadmhlo,Aaadmhol,almohada',
 'Estaban',
 'introduces,discounter',
 'percussion,supersonic',
 'Sevilla',
 'arrieros',
 'acertaron',
 'imaginaba',
 'represent',
 'chapiteles',
 'faltarle']

# Ejercicio N°6
UBER almacena en un cluster todos los datos sobre el movimiento y viajes de todos sus
vehículos. Existe un proceso que nos devuelve un RDD llamado trip_summary con los
siguientes campos: (driver_id, car_id, trip_id, customer_id, date (YYYYMMDD),
distance_traveled), Programar usando Py Spark un programa que nos indique cual fue el
conductor con mayor promedio de distancia recorrida por viaje para Abril de 2016.

In [267]:
datos = []
n = 1000
for i in range(n):
    driver_id = random.randint(1,25)
    car_id = driver_id
    trip_id = i
    customer_id = random.randint(1,n)
    date = "{}{}{}".format(random.randint(2015,2017)
                           ,random.choice(["01","02","03","04","05","06","07","08","09","10","11","12"])
                           ,random.choice(["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28"]))
    distance_traveled = random.randint(10,100)
    datos.append((driver_id, car_id, trip_id, customer_id, date, distance_traveled))
datosRDD = sc.parallelize(datos)

In [269]:
datosRDD.filter(lambda x: x[4] >= '20160401' and x[4] <= '20160428')\
        .map(lambda x: (x[0],(x[5],1)))\
        .reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))\
        .map(lambda x: (x[0],x[1][0]/x[1][1]))\
        .top(1, key=lambda x: x[1])

[(6, 100.0)]

# Ejercicio N°7
Una red social almacena el contenido de los chats entre sus usuarios en un RDD que tiene
registros con el siguiente formato: (chat_id, user_id, nickname, text). Queremos saber cuál es el
usuario (user_id) que mas preguntas hace en sus chats, contabilizamos una pregunta por cada
caracter “?” que aparezca en el campo text. Programar en Spark un programa que identifique al
usuario preguntón.

In [273]:
chatsRDD = sc.parallelize([(1,2,'bokita_el_mejor', '??????????????????????'), (2,2,'bokita_el_mejor', '??????'), (3,1,'ndeah','????????????????????????????????????????????????????????????????')])

In [275]:
chatsRDD.map(lambda x: (x[1], x[3].count('?')) )\
        .reduceByKey(lambda x,y: x+y)\
        .top(1, key= lambda x: x[1])

[(1, 64)]

# Ejercicio N°8
Contamos con un cluster que tiene 4 computadoras. Queremos aprovechar el paralelismo
del cluster para calcular los números primos entre 2 y 20.000.000. Para esto usaremos el
conocido algoritmo de la criba de Eratóstenes. Por ejemplo si empezamos con una lista de tipo
(2,3,4,5,6,7,8...) en un primer paso eliminamos todos los que son mayores a 2 y divisibles por 2
y nos queda (2,3,5,7,9,11,13...) luego eliminamos todos los mayores a 3 divisibles por 3 y nos
queda (2,3,5,7,11,13....etc) luego todos los divisibles por 5 y así sucesivamente. El resultado
final es una lista de números que son primos. Programar este programa usando PySpark.